In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

#pip install bs4
#pip install selenium
#pip install webdriver_manager
#pip install blinker==1.6.2

In [2]:
SCROLL_PAUSE_TIME = 3
pages = int(42)
cards = []

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
service = Service(ChromeDriverManager().install())

for i in range(1, pages+1):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    url = "https://www.zapimoveis.com.br/venda/apartamentos/go+goiania++setor-marista/?pagina=" + str(i)
    driver.get(url)

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        try:
            img_element = driver.find_element(By.XPATH, "//img[contains(@class, 'campaign__container-image')]")
            driver.execute_script("arguments[0].scrollIntoView();", img_element)
        except:
            pass
        
        time.sleep(SCROLL_PAUSE_TIME)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    anuncios = soup.find('div', class_="listing-wrapper__content").find_all('div', class_="BaseCard_card__content__pL2Vc w-full p-3")

    for anuncio in anuncios:
        card = {}
        lista_valores = anuncio.find_all('p')
        valor_limpo = [valor.text.strip() for valor in lista_valores]
        
        card['Endereço'] = valor_limpo[0]
        card['Preço'] = valor_limpo[1].replace("A partir de ", "")

        lista_carac = anuncio.find_all('li')
        carac_limpo = [carac.text.strip() for carac in lista_carac]

        card['Área (m2)'] = carac_limpo[0].replace(" m²", "")
        card['Quartos'] = carac_limpo[1]
        card['Banheiros'] = carac_limpo[2]
        card['Garagem'] = carac_limpo[3] if len(carac_limpo) > 3 else 0

        cards.append(card)

    driver.quit()


In [3]:
#pip install pandas
import pandas as pd

df = pd.DataFrame(cards)
df

,Endereço,Preço,Área (m2),Quartos,Banheiros,Garagem
0,Rua 1124,R$ 1.929.000,157,3,4,2
1,Avenida T 9,R$ 2.355.217,185-265,1-5,1-5,0-2
2,Rua 1141,R$ 1.090.000,111,2,4,2
3,Rua 1124,R$ 2.190.000,157,3,4,2
4,Rua 1123,R$ 370.000,69,2,2,1
...,...,...,...,...,...,...
4347,,R$ 4.600.000,306,4,6,5
4348,,R$ 1.230.000,155,3,4,2
4349,,R$ 1.180.000,111,3,5,2
4350,,R$ 1.990.000,179,3,5,3


In [4]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4352 entries, 0 to 4351
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Endereço   4352 non-null   object
 1   Preço      4352 non-null   object
 2   Área (m2)  4352 non-null   object
 3   Quartos    4352 non-null   object
 4   Banheiros  4352 non-null   object
 5   Garagem    4352 non-null   object
dtypes: object(6)
memory usage: 204.1+ KB


None

In [5]:
df.value_counts()

Endereço  Preço         Área (m2)  Quartos  Banheiros  Garagem
Rua 138   R$ 1.750.000  214        4        5          3          25
Rua 143   R$ 930.000    115        3        7          2          22
Rua 1141  R$ 1.990.000  256        4        5          3          17
Rua 1124  R$ 1.500.000  125        3        4          2          15
Rua 38    R$ 1.600.000  133        3        4          2          14
                                                                  ..
          R$ 1.080.000  130        3        4          3           1
          R$ 1.074.268  109        3        4          2           1
          R$ 1.050.000  115        3        3          2           1
                        111        3        4          2           1
                        106        3        2          2           1
Name: count, Length: 2291, dtype: int64

In [6]:
df = df.drop_duplicates()
df

,Endereço,Preço,Área (m2),Quartos,Banheiros,Garagem
0,Rua 1124,R$ 1.929.000,157,3,4,2
1,Avenida T 9,R$ 2.355.217,185-265,1-5,1-5,0-2
2,Rua 1141,R$ 1.090.000,111,2,4,2
3,Rua 1124,R$ 2.190.000,157,3,4,2
4,Rua 1123,R$ 370.000,69,2,2,1
...,...,...,...,...,...,...
4317,,R$ 1.990.000,179,3,5,3
4318,,R$ 2.200.000,208,3,4,3
4319,,R$ 1.420.000,130,3,3,2
4320,,R$ 4.689.000,317,4,6,4


In [7]:
df.to_csv("ApartamentosMarista.csv", index=False, sep=";", encoding="latin1")